In [42]:
from dotenv import load_dotenv
import os
import pandas as pd
import openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.chains import LLMChain
from typing import Union, List ,Optional
from pydantic import BaseModel, Field
from typing_extensions import Annotated, TypedDict

In [31]:
load_dotenv(r"C:/Users/helin/source/repos/opengll/gpt/api.env")

True

In [32]:
llm = ChatOpenAI(model="gpt-4o-mini")

# read menu

In [1]:


# Menü dosyasını okuma
def read_menu(file_path):
    menu = pd.read_csv(file_path)
    return menu

# Menü verisini yapay zekaya uygun bir formata dönüştürme
def prepare_menu_for_gpt(menu):
    menu_items = []
    for _, row in menu.iterrows():
        menu_items.append({
            "name": row["item_name"],
            "description": row["description"],
            "ingredients": row["ingredients"],
            "price": row["price"]
        })
    return menu_items

# Menü örneği
file_path = "menu.csv"  # Menü CSV dosyasının yolu
menu = read_menu(file_path)
menu_for_gpt = prepare_menu_for_gpt(menu)

# Menü bilgisi GPT'ye gönderilebilir
print(menu_for_gpt)


[{'name': 'Burger', 'description': 'Beef patty with lettuce and tomato', 'ingredients': 'beef,lettuce,tomato,bun', 'price': 8.99}, {'name': 'Pizza', 'description': 'Classic cheese pizza', 'ingredients': 'cheese,tomato sauce,flour', 'price': 12.99}, {'name': 'Pasta', 'description': 'Creamy Alfredo pasta', 'ingredients': 'pasta,cream,parmesan', 'price': 10.99}]


# System Prompt

In [38]:
system_prompt = (
    "You are a professional waiter at a high-quality restaurant. "
    "Your job is to politely and concisely answer questions about the menu. "
    "Always start your response with a warm greeting, such as 'Welcome!' or 'Hello, how can I assist you today?'. "
    "Use the provided menu context to answer all questions. "
    "If the item is not available, respond with: 'I’m sorry, that item is not on the menu.' "
    "If you don't know the answer, say: 'I’m sorry, I’m not sure about that.' "
    "When confirming an order, repeat it politely and thank the guest. "
    "Do not recommend items outside the provided menu. "
    "Structure your responses as follows: "
    "1. Start with a warm greeting. "
    "2. Provide a clear and concise answer to the question. "
    "3. Close with a polite and professional offer to assist further. "
    "Only response the main question. Don't say anything except the question's response. "
    "\n\n"
    "{context}"
)

# ChatPromptTemplate
prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)


# Create the question-answer chain
#question_answer_chain = create_stuff_documents_chain(llm, prompt)
llm_chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\helin\AppData\Local\Temp\ipykernel_3820\1509826299.py:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [43]:


class sip(TypedDict):
    """Represents a sentence describing an ordered item."""

    sip_name: Annotated[str, ..., "Name of the ordered item, such as a menu item."]
    sip_count: Annotated[int, ..., "Quantity of the ordered item."]

class non_sip(TypedDict):
    """Represents a sentence describing a non-ordered item."""

    non_sip_name: Annotated[str, ..., "Name of the item, without an order (e.g., general reference to a thing or asking questions about it)."]
    non_sip_count: Annotated[int, ..., "Count of the item, if mentioned, but not part of an order."]

class del_item(TypedDict):
    """Represents a sentence describing an item to be deleted from the order."""
    
    del_name: Annotated[str, ..., "Name of the ordered item that is being canceled or removed."]
    del_count: Annotated[int, ..., "Quantity of the item that is being canceled or removed. This value can be positive or indicate the total amount to be canceled."]
    is_all: Annotated[Optional[bool], "Indicates if the entire item category (e.g., all 'hamburgers') is being canceled. 'True' means all instances of that item will be canceled, 'False' means just a specific quantity or instance."]
    description: Annotated[Optional[str], "Describes the user's intent to cancel all instances of a specific item category, like 'hamburgers', without providing a specific count. This could include phrases like 'I want to cancel all hamburgers'."]

class FinalResponse(BaseModel):
    final_output: List[Union[sip, non_sip, del_item]]

# Example of structured output model usage
structured_llm = llm.with_structured_output(FinalResponse)




In [39]:
system1 = """You are an expert at converting user questions into database queries. \
customers will talk you about the menu and you will tke the order or answer their questions \
you have to understand the input for deciding to take an order or answer the question \
if you dont understand the meaning of a sentence or the intent tell it """

prompt1 = ChatPromptTemplate.from_messages(
    [
        ("system", system1),
        ("human", "{query}"),
    ]
)

In [44]:
structured_llm1 = llm.with_structured_output(FinalResponse)
query_analyzer = prompt1 | structured_llm1

Error: Windows is not supported yet in the migration CLI
